In [52]:
!pip install packaging==23.2
!pip install langchain langchain-ollama openai python-dotenv
!pip install langchain-community
!pip install langchain-ollama
!pip install langchain-text-splitters

In [50]:
import os
import requests
from textwrap import dedent
from IPython.display import Markdown
import langchain

# Initialize Ollama
from langchain_ollama import OllamaLLM
from langchain_community.llms import Ollama
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain

In [51]:
# Configure Ollama model with the local URL
llm = OllamaLLM(model="llama2", api_url="http://localhost:11434")

In [38]:
# Store your API key for Tavily securely
os.environ["TAVILY_API_KEY"] = "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"  # Make sure this key is correct!

# Function to interact with Tavily API
def tavily_search(query):
    headers = {
        "Authorization": f"Bearer {os.getenv('TAVILY_API_KEY')}"
    }
    params = {
        "query": query,
        "search_depth": "advanced",
        "include_answers": "true",
        "include_raw_content": "false",
        "max_results": 5
    }
    response = requests.post("https://api.tavily.com/search", headers=headers, json=params)
    data = response.json()

    print("Tavily API Response:", data)  # Log the response for debugging

    if 'results' in data:
        return data["results"]
    else:
        return []

# Function to format the research results
def research_agent(query):
    results = tavily_search(query)
    summaries = []
    for item in results:
        summaries.append(f"{item['title']}: {item['content'][:300]}... ({item['url']})")
    return "\n".join(summaries)


In [39]:
# Configure Ollama
llm = Ollama(model="llama2", temperature=0.7)

# Prompt template with clear formatting instructions
template = """
You are an expert research analyst. Based on the following research results:
{research_results}
Please provide a well-structured response with these sections:
1. Key Findings (bullet points)
2. Detailed Analysis (2-3 paragraphs)
3. Top References (formatted as [Title](URL))
Format the response in Markdown for better readability.
"""

prompt = PromptTemplate(
    input_variables=["research_results"],
    template=template
)
chain = LLMChain(llm=llm, prompt=prompt)

In [40]:
def tavily_search(query):
    headers = {"Authorization": f"Bearer {os.getenv('TAVILY_API_KEY')}"}
    params = {
        "query": query,
        "search_depth": "advanced",
        "include_answers": True,
        "include_raw_content": False,
        "max_results": 5
    }
    response = requests.post("https://api.tavily.com/search", headers=headers, json=params)
    return response.json()

In [41]:

def format_research_results(results):
    formatted = []
    
    # Add Tavily's answer if available
    if 'answer' in results and results['answer']:
        formatted.append(f"## Tavily Summary\n{results['answer']}\n")
    
    # Format individual results
    for item in results.get('results', []):
        formatted.append(dedent(f"""
        - **Title**: {item.get('title', 'N/A')}
          **URL**: [{item.get('url', 'N/A')}]({item.get('url', 'N/A')})
          **Content**: {item.get('content', 'N/A')[:200]}...
          **Relevance Score**: {item.get('score', 0):.2f}
        """))
    return "\n".join(formatted)

In [42]:
def complete_research(query):
    """
    Complete research and return results as Markdown
    """
    # Get and format research
    raw_results = tavily_search(query)
    research_results = format_research_results(raw_results)
    
    # Generate final answer
    response = chain.invoke({"research_results": research_results})
    
    # Post-process for consistent formatting
    formatted_response = (
        response['text']
        .replace('•', '-')  # Standardize bullet points
        .replace('\n\n\n', '\n\n')  # Remove excessive newlines but keep paragraph breaks
    )
    
    # Add header and query info
    final_output = f"# Research Results: {query}\n\n{formatted_response}"
    
    # Return as Markdown for display
    return Markdown(final_output)

# Usage example
query = "What are the key trends in AI technology?"
complete_research(query)

# Research Results: What are the key trends in AI technology?

Key Findings:

* AI trends in 2025 will focus on improving decision-making and increasing accessibility and accuracy of data-driven decisions.
* Generative AI is becoming more expensive, error-prone, and vulnerable to cyber attacks, making it a challenge for companies to adopt.
* Companies are looking for proven results from generative AI rather than early-stage prototypes.
* The use of AI in the technology industry is growing, with 73% of US companies using AI in some capacity.
* Eight important AI trends to watch in 2025 include: genetic AI, machine learning, natural language processing, robotics, computer vision, autonomous vehicles, and edge computing.

Detailed Analysis:
AI trends in 2025 will prioritize improving decision-making processes through the use of AI tools. These tools aim to make data-driven decisions more accessible and accurate, with a focus on genetic AI, machine learning, natural language processing, robotics, computer vision, autonomous vehicles, and edge computing. However, generative AI is becoming increasingly expensive, error-prone, and vulnerable to cyber attacks, making it challenging for companies to adopt. As such, companies are looking for proven results from generative AI rather than early-stage prototypes. The use of AI in the technology industry is growing, with 73% of US companies using AI in some capacity.

Top References:

1. [14 AI Trends Transforming Tech in 2025 and Beyond](https://www.digitalocean.com/resources/articles/ai-trends)
2. [8 AI and Machine Learning Trends to Watch in 2025](https://www.techtarget.com/searchenterpriseai/tip/9-top-AI-and-machine-learning-trends)
3. [Top 5 AI Trends to Watch in 2025](https://www.coursera.org/articles/ai-trends)
4. [AI Trends for 2025 - YouTube](https://www.youtube.com/watch?v=5zuF4Ys1eAw&pp=0gcJCdgAo7VqN5tD)
5. [6 AI trends you'll see more of in 2025 - Microsoft News](https://news.microsoft.com/source/features/ai/6-ai-trends-youll-see-more-of-in-2025/)

In [43]:
#example 2

query = "What are the key trends in defence technology?"
complete_research(query)

# Research Results: What are the key trends in defence technology?

Key Findings:

* The aerospace and defense industry is witnessing significant technological advancements in 2025, with the application of artificial intelligence, immersive technologies, additive manufacturing, and other emerging technologies gaining traction.
* The industry is expected to undergo a digital transformation, leading to increased automation, efficiency, and innovation.
* Key trends driving innovation in the defense industry include AI, space technology, cybersecurity, sustainability, and innovations shaping the future of global security.

Detailed Analysis:
The aerospace and defense industry is experiencing a technological revolution in 2025, with various emerging trends transforming the sector. The application of artificial intelligence (AI) is one of the most significant advancements, enabling defense organizations to analyze vast amounts of data, make more informed decisions, and improve operational efficiencies. Immersive technologies such as virtual and augmented reality are also gaining traction, providing innovative solutions for training, simulation, and other applications.

Additive manufacturing (3D printing) is another technology that is transforming the defense industry. It enables the production of complex parts with increased speed, efficiency, and reduced costs. The use of quantum radar is also expected to revolutionize air defense systems, providing unparalleled detection capabilities.

The industry is also witnessing a growing focus on cybersecurity, as defense organizations recognize the importance of protecting their digital assets from emerging threats. Sustainability is another key trend, with defense organizations increasingly focusing on reducing their environmental footprint and developing more eco-friendly technologies.

Top References:

* [Title] New Technology Trends in Aerospace and Defense Industry [2025](https://www.epicflow.com/blog/driving-digital-transformation-in-aerospace-defense-recent-technology-trends/)
* [Title] Defence 2025: Key Trends Driving Innovation](https://mssdefence.com/blog/defence-2025-key-trends-driving-innovation/)
* [Title] 2025 Defense Industry Trends - Free Report - ITONICS](https://www.itonics-innovation.com/defense-trends)
* [Title] 5 Key Trends Shaping the Future of Defense Technology in 2025](https://www.linkedin.com/pulse/5-key-trends-shaping-future-defense-technology-2025-david-schunk-lubhe)
* [Title] Defense and security trends 2025: Building strength with data and AI](https://www.the-future-of-commerce.com/2025/02/01/defense-and-security-trends-2025/)